In [ ]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
assert os.path.basename(os.getcwd()) == "element-moseq", (
    "Please move to the " + "element directory"
)

In [ ]:
import datajoint as dj
from pathlib import Path

In [ ]:
dj.config.load("dj_local_conf.json")

In [ ]:
dj.conn()

In [ ]:
# dj.schema('vathes-team_devlab_milagros_moseq_testing3_pca').drop()

In [ ]:
from tutorial_pipeline import lab, subject, session, pca, model, Device

In [ ]:
# pca.KeypointSet.VideoFiles.delete_quick()
# pca.KeypointSet.PoseEstimationMethod.delete_quick()
# pca.PCAInfo.delete_quick()
# pca.KeypointSet.delete()

In [ ]:
dj.Diagram(pca)
#+ dj.Diagram(model)

In [ ]:
pca.KeypointSet()

In [ ]:
# Subject and Session tables
subject.Subject.insert1(
    dict(
        subject="subject1",
        sex="F",
        subject_birth_date="2020-01-01",
        subject_description="test",
    ),
    skip_duplicates=True,
)

#Definition of the dictionary named "session_keys"
session_keys = [
    dict(subject="subject1", session_datetime="2021-06-02 14:04:22"),
    dict(subject="subject1", session_datetime="2021-06-03 14:43:10"),
]

#Insert this dictionary in the Session table
session.Session.insert(session_keys, skip_duplicates=True)

In [ ]:
kpset_key = dict(subject="subject1",
            session_datetime="2021-06-02 14:04:22",
            kpset_id=1)
pca.KeypointSet.insert1({**kpset_key, 
                            "kpset_path":"/input_data/videos",
                            "kpset_description":"testing pca schema"},
                            skip_duplicates=True)

In [ ]:
pca.KeypointSet()

In [ ]:
video_files = ["/input_data/videos/21_11_8_one_mouse.top.ir.Mp4",
               "/input_data/videos/21_12_2_def6a_1.top.ir.mp4",
               "/input_data/videos/21_12_2_def6b_2.top.ir.mp4"]

pca.KeypointSet.VideoFiles.insert(
    ({**kpset_key, "video_id":v_idx, "video_path":Path(f)}
    for v_idx, f in enumerate(video_files)),
    skip_duplicates=True
)

In [ ]:
pca.KeypointSet.VideoFiles()

In [ ]:
pca.KeypointSet.PoseEstimationMethod.insert1(
    {**kpset_key,
     "format":"deeplabcut",
     "extension":"h5",
     "device":"Camera1"},
     skip_duplicates=True
)

In [ ]:
pca.KeypointSet.PoseEstimationMethod()

In [ ]:
# pca.RecordingInfo.populate()

In [ ]:
pca.Bodyparts()

In [ ]:
# Insert data in PCAInfo table
bodypart_key = {**kpset_key,
                "bodyparts_id":1}
pca.Bodyparts.insert1({**bodypart_key,
                "anterior_bodyparts":"nose",
                "posterior_bodyparts":"spine4",
                "use_bodyparts":['spine4', 'spine3', 'spine2', 'spine1',
                'head', 'nose', 'right ear', 'left ear']},
                    skip_duplicates=True)



In [ ]:
pca.DataLoadingTask.insert1({**bodypart_key,
                            "task_mode":"load"},
                            skip_duplicates=True)

In [ ]:
pca.DataLoadingTask()

In [ ]:
pca.DataLoadingTask.generate(bodypart_key)

In [ ]:
pca.DataLoadingTask.FormattedDataset()